<a href="https://colab.research.google.com/github/andyly13/clause-extraction/blob/main/clause_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2024-02-07 10:06:51--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2024-02-07 10:06:52--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.16K  --.-KB/s    in 0s      

2024-02-07 10:06:52 (130 MB/s) - written to stdout [1191/1191]

Installing 

In [ ]:
import sparknlp
spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 5.2.3
Apache Spark version: 3.2.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pikepdf
from pikepdf import Pdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 29.2 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.2.0 which is incompatible.


In [ ]:
# Andrew Ly Clause Extraction Code (02/07/2024)
# getting all of the longest files in each folder in sorted

!pip install PyPDF2

from multiprocessing import Pool
import PyPDF2
import os

# ----- functions -------

# returns a generator that holds all of the ID folders
def contract_generator(root_directory):
    contract_folders = os.listdir(root_directory)
    for id in contract_folders:
        contracts = os.listdir(os.path.join(root_directory, id))
        yield contracts, id

# appends the longest contract to the pdfs_to_check list to be used for text-conversion?
def add_pdfs(root_directory, longest_contract_dict):
    contract_gen = contract_generator(root_directory)
    while True:
      try:
        contracts, id = next(contract_gen)

        if len(contracts) == 1:
          longest_contract_dict[int(id)] = contracts[0] # add first and only pdf
          print("Contract " + contracts[0] + " added from file " + id)
        elif len(contracts) == 0: # handling empty folder, think there was only one but it broke everything LOL
          longest_contract_dict[int(id)] = 'None'
          print("File " + id + " was empty. Added 'None' to dict.")
        else:
          longest_contract = get_longest_contract(root_directory, contracts, id)
          longest_contract_dict[int(id)] = longest_contract
          print("Contract " + longest_contract + " added from file " + id)
      except StopIteration: # occurs when generator gets to the end of the file
        break

# takes a list of contracts, returns the one with the most pages
def get_longest_contract(root_directory, contracts, id):
  max_pages = 0
  longest_contract = contracts[0]

  with Pdf.open(os.path.join(root_directory, id, contracts[0])) as longest_pdf:
    max_pages = len(longest_pdf.pages)

  for i in range(1, len(contracts)):
    with Pdf.open(os.path.join(root_directory, id, contracts[i])) as temp_pdf:
      temp_pages = len(temp_pdf.pages)

      if temp_pages > max_pages:
        longest_contract = contracts[i]
        max_pages = temp_pages

  # with open(os.path.join(root_directory, id, contracts[0]), 'rb') as fileObj:
  #   pdfReader = PyPDF2.PdfReader(fileObj)
  #   if pdfReader.is_encrypted:
  #     return 'None (Encrypted)' # skipping for now
  #   else:
  #     max_pages = len(pdfReader.pages)

  # # gets longest contract
  # for contract in contracts:
  #   with open(os.path.join(root_directory, id, contract), 'rb') as fileObj:
  #     pdfReader = PyPDF2.PdfReader(fileObj)
  #     if len(pdfReader.pages) > max_pages:
  #       longest_contract = contract
  #       max_pages= len(pdfReader.pages)

  return longest_contract


# ----- calls --------

longest_contract_dict = {} # folder ID is the key and contract path is the value
root_directory = '/content/drive/Shareddrives/Contract Data/Contracts (PDF)/1 Sorted'

with Pool() as pool: # not sure if this does anything, but should help it run faster using multiple CPU cores
  add_pdfs(root_directory, longest_contract_dict)




Contract Deal_167694_Research_License_Collaboration1.pdf added from file 167694
Contract Deal_167518_Asset Purchase1.pdf added from file 167518
Contract Deal_167623_Acquisition1.pdf added from file 167623
Contract Deal_167464_Distribution_Collaboration1.pdf added from file 167464
Contract Deal_167254_Acquisition1.pdf added from file 167254
Contract Deal_167429_Loan2.pdf added from file 167429
Contract Deal_167402_Supply_License1.pdf added from file 167402
Contract Deal_167416_Joint Venture2.pdf added from file 167416
Contract Deal_167322_Amendment1.pdf added from file 167322
Contract Deal_167371_Amendment2.pdf added from file 167371
Contract Deal_167361_Asset Purchase5.pdf added from file 167361
Contract Deal_167302_License3.pdf added from file 167302
Contract Deal_167208_contract1.pdf added from file 167208
Contract Deal_167147_contract2.pdf added from file 167147
Contract Deal_167222_Registration1.pdf added from file 167222
Contract Deal_167241_contract1.pdf added from file 167241
Co

In [ ]:
# Andrew Ly - importing dictionary to CSV
# Longest contract spreadsheet is viewable in google sheets

import csv

def export_to_csv(data, file_name):
    with open(file_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['ID', 'Longest Contract'])  # Write header
        for id, contract in data.items():
            writer.writerow([id, contract])

export_to_csv(longest_contract_dict, 'contracts.csv')


In [ ]:
# Andrew Ly - Creating new file directory with longst contracts
# Will create new directory in drive named 'Longest Contracts'


